In [ ]:
import ipywidgets as widgets
start_date = widgets.DatePicker(
    description='Select a start date',
    disabled=False,
    style={'description_width': 'initial'}
)
end_date = widgets.DatePicker(
    description='Select a end date',
    disabled=False,
    style={'description_width': 'initial'}
)

display(start_date)
display(end_date)

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import plotly.express as px
URL = "https://earthquake.usgs.gov/fdsnws/event/1/query.csv?starttime=" + str(start_date.value) + "%2000:00:00&endtime=" + str(end_date.value) + "%2023:59:59&maxlatitude=50&minlatitude=24.6&maxlongitude=-65&minlongitude=-125&minmagnitude=2.5&orderby=time"
df = pd.read_csv(URL)

fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", color="mag",hover_name="place", hover_data=["nst", "mag"],
                        color_continuous_scale=px.colors.cyclical.IceFire, zoom=3, height=600,
                        title="Earthquakes Across the United States",
                        template='plotly_dark')
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


fig = px.scatter_3d(
    df,
    title='Earthquake Depth, Magnitude, and Magnitude Error',
    x="mag",
    y="depth",
    z="depthError",
    color="magError",
    template='plotly_dark',
    height=600,
)
fig.update_layout(showlegend=False)
fig.show()


fig = px.histogram(df[df['nst']<=100], x="nst",template='plotly_dark',title='Number of Sensors Used per Earthquake',)
fig.show()

def map_robinson(df, color, animation_frame):
# country = px.data.gapminder()
    fig = px.scatter_geo(df, lat="latitude", lon="longitude", 
                         hover_name="place", color = color,
                         animation_frame = animation_frame,
                         projection="natural earth",
                         height = 900,
                         template='plotly_dark',
                         scope='north america',
                         title="Location and Depth of Earthquakes across time, between " + str(start_date.value) + " and " + str(end_date.value) 
                         )                     
    fig.show()

# Converting time column to datetime obj and splitting it up into year, month and day to potentially use it as filters later.
df['time'] = pd.to_datetime(df['time'])
df['year'] = pd.DatetimeIndex(df['time']).year
df['month'] = pd.DatetimeIndex(df['time']).month
df['day'] = pd.DatetimeIndex(df['time']).day

#Splitting column based on comma to see if it actually contains only state names that can also be used later
df['location_after_comma'] = df['place'].str.rsplit(',').str[-1]


map_robinson(df, "depth", "day")